In [1]:
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)

In [2]:
from scapy.all import *

# Send packets

Let's create some packets first:

In [3]:
pkt1 = IP(dst="www.python.org/30")

In [4]:
pkt1

<IP  dst=Net('www.python.org/30') |>

In [5]:
[p for p in pkt1]

[<IP  dst=103.245.222.220 |>,
 <IP  dst=103.245.222.221 |>,
 <IP  dst=103.245.222.222 |>,
 <IP  dst=103.245.222.223 |>]

In [6]:
pkt2 = IP(ttl=[1,2,(5,9)])

In [7]:
pkt2

<IP  ttl=[1, 2, (5, 9)] |>

In [8]:
[p for p in pkt2]

[<IP  ttl=1 |>,
 <IP  ttl=2 |>,
 <IP  ttl=5 |>,
 <IP  ttl=6 |>,
 <IP  ttl=7 |>,
 <IP  ttl=8 |>,
 <IP  ttl=9 |>]

In [9]:
pkt3 = TCP(dport=[80,443])

In [11]:
pkts = [p for p in pkt1/pkt3]

In [12]:
pkts

[<IP  frag=0 proto=tcp dst=103.245.222.220 |<TCP  dport=http |>>,
 <IP  frag=0 proto=tcp dst=103.245.222.220 |<TCP  dport=https |>>,
 <IP  frag=0 proto=tcp dst=103.245.222.221 |<TCP  dport=http |>>,
 <IP  frag=0 proto=tcp dst=103.245.222.221 |<TCP  dport=https |>>,
 <IP  frag=0 proto=tcp dst=103.245.222.222 |<TCP  dport=http |>>,
 <IP  frag=0 proto=tcp dst=103.245.222.222 |<TCP  dport=https |>>,
 <IP  frag=0 proto=tcp dst=103.245.222.223 |<TCP  dport=http |>>,
 <IP  frag=0 proto=tcp dst=103.245.222.223 |<TCP  dport=https |>>]

And let's send 'em!  `send()` sends packets on the 3rd link layer (e.g. IP, ARP, etc):

In [13]:
send(pkts[0])


Sent 1 packets.


And `sendp` sends on the 2nd link layer (Ethernet, 802.3, etc):

In [14]:
sendp(pkts[1])


Sent 1 packets.


In [15]:
sendp(Ether()/IP(dst="1.2.3.4", ttl=(1,4)), iface="en0")  # iface is en# for Mac, eth# in Linux


Sent 4 packets.


### Now let's send & receive some packets!

There's a few different functions we can use, all very similar.  First we'll look at `sr1` - which just returns the first answered packet:

In [47]:
answered = sr1(IP(dst="103.245.222.223")/ICMP()/"FooBar")


Received 2 packets, got 1 answers, remaining 0 packets
Begin emission:
Finished to send 1 packets.


In [48]:
answered

<IP  version=4L ihl=5L tos=0x0 len=34 id=50614 flags= frag=0L ttl=54 proto=icmp chksum=0xdc4c src=103.245.222.223 dst=10.121.145.138 options=[] |<ICMP  type=echo-reply code=0 chksum=0xe8db id=0x0 seq=0x0 |<Raw  load='FooBar' |<Padding  load='\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' |>>>>

In [49]:
answered.show()

###[ IP ]###
  version   = 4L
  ihl       = 5L
  tos       = 0x0
  len       = 34
  id        = 50614
  flags     = 
  frag      = 0L
  ttl       = 54
  proto     = icmp
  chksum    = 0xdc4c
  src       = 103.245.222.223
  dst       = 10.121.145.138
  \options   \
###[ ICMP ]###
     type      = echo-reply
     code      = 0
     chksum    = 0xe8db
     id        = 0x0
     seq       = 0x0
###[ Raw ]###
        load      = 'FooBar'
###[ Padding ]###
           load      = '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


Now `sr` which will collect all answered and unanswered packets we send:

In [54]:
ans, unans = sr(IP(dst="103.245.222.223")/TCP(dport=[80,443]))


Received 7 packets, got 2 answers, remaining 0 packets
Begin emission:
Finished to send 2 packets.


In [55]:
ans

<Results: TCP:2 UDP:0 ICMP:0 Other:0>

In [56]:
ans.summary()

IP / TCP 10.121.145.138:ftp_data > 103.245.222.223:http S ==> IP / TCP 103.245.222.223:http > 10.121.145.138:ftp_data SA / Padding
IP / TCP 10.121.145.138:ftp_data > 103.245.222.223:https S ==> IP / TCP 103.245.222.223:https > 10.121.145.138:ftp_data SA / Padding
